# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

C. Clontz  ->  C. Clontz  |  ['C. Clontz']
M. Häberle  ->  M. Häberle  |  ['M. Häberle']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
J. Liu  ->  J. Liu  |  ['J. Liu']
S. Li  ->  S. Li  |  ['S. Li']
J. Lian  ->  J. Lian  |  ['J. Lian']


Arxiv has 72 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.13855


extracting tarball to tmp_2409.13855...

 done.
  0: tmp_2409.13855/main.tex, 897 lines
  1: tmp_2409.13855/oMEGACat_IV_SGB_ages_arxiv/main.tex, 891 lines


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2409.13855/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


...

S. Li  ->  S. Li  |  ['S. Li']


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure figures/myXCLASSMap_OCS.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\textwidth]{figures/myXCLASSMap_OCS.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 64 bibliographic references in tmp_2409.14445/ms.bbl.
Error retrieving bib data for 2001Lee_jet_wind_model: 'author'
Error retrieving bib data for 2001Lee_jet_wind_model: 'author'
Retrieving document from  https://arxiv.org/e-print/2409.14716


extracting tarball to tmp_2409.14716...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.13855-b31b1b.svg)](https://arxiv.org/abs/2409.13855) | **oMEGACat IV: Constraining Ages of Omega Centauri sub-giant branch stars with HST and MUSE**  |
|| <mark>C. Clontz</mark>, et al. -- incl., <mark>M. Häberle</mark>, <mark>N. Neumayer</mark> |
|*Appeared on*| *2024-09-24*|
|*Comments*| *23 pages, 11 figures*|
|**Abstract**|            We present age estimates for over 8100 sub-giant branch (SGB) stars in Omega Centauri ($\omega$ Cen) to study its star formation history. Our large data set, which combines multi-wavelength HST photometry with MUSE metallicities, provides an unprecedented opportunity to measure individual stellar ages. We do this by fitting each star's photometry and metallicity with theoretical isochrones, that are embedded with an empirical [C+N+O]-[Fe/H] relation specifically for $\omega$ Cen. The bulk of the stars have ages between 13 and 10 Gyr, with the mean stellar age being 12.08$\pm$0.01 Gyrs and the median age uncertainty being 0.68 Gyrs. From these ages we construct the most complete age-metallicity relation (AMR) for $\omega$ Cen to date. We find that the mean age of stars decreases with increasing metallicity and find two distinct streams in the age-metallicity plane, hinting at different star formation pathways. We derive an intrinsic spread in the ages of 0.75$\pm$0.01 Gyr for the whole cluster, with the age spread showing a clear increase with metallicity. We verify the robustness of our age estimations by varying isochrone parameters and constraining our systematics. We find the C+N+O relation to be the most critical consideration for constraining the AMR. We also present the SGB chromosome map with age information. In the future, these stellar ages could be combined with chemical abundances to study age differences in subpopulations, and uncover the chemical evolution history of this massive nuclear star cluster.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.14445-b31b1b.svg)](https://arxiv.org/abs/2409.14445) | **ALMASOP. The Localized and Chemically rich Features near the Bases of the Protostellar Jet in HOPS 87**  |
|| S.-Y. Hsu, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-09-24*|
|*Comments*| *16 pages, 6+2 figures, accepted by ApJ*|
|**Abstract**|            HOPS 87 is a Class 0 protostellar core known to harbor an extremely young bipolar outflow and a hot corino. We report the discovery of localized, chemically rich regions near the bases of the two-lobe bipolar molecular outflow in HOPS 87 containing molecules such as H$_2$CO, $^{13}$CS, H$_2$S, OCS, and CH$_3$OH, the simplest complex organic molecule (COM). The locations and kinematics suggest that these localized features are due to jet-driven shocks rather than being part of the hot corino region encasing the protostar. The COM compositions of the molecular gas in these jet-localized regions are relatively simpler than those in the hot corino zone. We speculate that this simplicity is due to either the liberation of ice with a less complex chemical history or the effects of shock chemistry. Our study highlights the dynamic interplay between the protostellar bipolar outflow, disk, inner core environment, and the surrounding medium, contributing to our understanding of molecular complexity in solar-like young stellar objects.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.14716-b31b1b.svg)](https://arxiv.org/abs/2409.14716) | **Simultaneous Multiband Photometry of the Early Optical Afterglow of GRB 240825A with Mephisto**  |
|| Y. Cheng, et al. -- incl., <mark>J. Lian</mark> |
|*Appeared on*| *2024-09-24*|
|*Comments*| *15 pages, 5 figures, 1 table. Comments welcome!*|
|**Abstract**|            Gamma-ray bursts (GRBs) are the most luminous transients in the universe. The interaction of the relativistic jet with the circumburst medium produces an afterglow and generates multiwavelength emission. In this work, we present simultaneous multiband photometry of GRB~240825A with the Multi-channel Photometric Survey Telescope (Mephisto) and analyze its temporal and spectral properties. The measurement began 128 seconds after the GRB trigger and continued until the fourth day when the afterglow essentially diminished and the measured brightness was close to that of the host galaxy. Based on the multiband light curves in the $uvgriz$ bands, we find that the optical flux density satisfies $F_{\nu,{\rm obs}}\propto t^{-1.34}\nu^{-2.48}$ with a spectral index of $2.48$ much larger than those of most other GRBs. To reconcile the measured much softer spectral energy distribution (SED) with that predicted by the standard afterglow model, an extra host-galaxy extinction of $E_{B-V}\sim(0.37-0.57)$ mag is required. We interpreted this excess as arising from a dense circumburst medium. We further find that the SED of the optical afterglow hardened as the afterglow decayed and the color excess $E_{B-V}$ decreased $\sim0.21$ mag in the first 3000 seconds. Finally, we analyze the properties of the host galaxy of GRB~240825A based on data from the SDSS, PanSTARRS and HSC-SSP surveys. For a host redshift of $z=0.659$, the stellar mass and star formation rate of the host galaxy are estimated to be $\log(M_*/M_\odot)=10.0^{+0.3}_{-0.3}$ and $\log({\rm SFR}/M_{\odot}{\rm yr}^{-1})= 0.6^{+0.8}_{-3.3}$, respectively, pointing to a gas-rich, star-forming, medium-size galaxy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.14382-b31b1b.svg)](https://arxiv.org/abs/2409.14382) | **Layout optimization and Performance of Large Array of imaging atmospheric Cherenkov Telescope (LACT)**  |
|| Z. Zhang, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2024-09-24*|
|*Comments*| *24 pages, 19 figures, submited to Chinese Physcis C*|
|**Abstract**|            Large Array of imaging atmospheric Cherenkov Telescope (LACT) is an array of 32 Cherenkov telescopes with 6-meter diameter mirrors to be constructed at the LHAASO site. In this work, we present a study on the layout optimization and performance analysis of LACT. We investigate two observation modes: large zenith angle observations for ultra-high energy events and small zenith angle observations for lower energy thresholds. For large zenith angles (60°), simulations show that an 8-telescope subarray can achieve an effective area of $3 ~\rm km^2$ and excellent angular resolution. For small zenith angles, we optimize the layout of 4-telescope cells and the full 32-telescope array. The threshold of the full array is about $200~\rm GeV$, which is particularly crucial for studying transient phenomena, including gamma-ray bursts (GRBs) and active galactic nuclei (AGNs). This study provides important guidance for the final LACT layout design and performance estimates under different observational conditions, demonstrating LACT's potential for deep observations of ultra-high energy \gray sources and morphological studies of PeVatrons, as well as time-domain \gray astronomy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.13855/./figures/mean_ages_and_age_spreads_systematics.png', 'tmp_2409.13855/./figures/amr_two_panels.png', 'tmp_2409.13855/./figures/sgb_polygon_selection.png']
copying  tmp_2409.13855/./figures/mean_ages_and_age_spreads_systematics.png to _build/html/
copying  tmp_2409.13855/./figures/amr_two_panels.png to _build/html/
copying  tmp_2409.13855/./figures/sgb_polygon_selection.png to _build/html/
exported in  _build/html/2409.13855.md
    + _build/html/tmp_2409.13855/./figures/mean_ages_and_age_spreads_systematics.png
    + _build/html/tmp_2409.13855/./figures/amr_two_panels.png
    + _build/html/tmp_2409.13855/./figures/sgb_polygon_selection.png
found figures ['tmp_2409.14445/./figures/mom0_transs.png', 'tmp_2409.14445/./figures/PVtranss.png', 'tmp_2409.14445/./figures/chn_CH3OH_Cycle3.png']
copying  tmp_2409.14445/./figures/mom0_transs.png to _build/html/
copying  tmp_2409.14445/./figures/PVtranss.png to _build/html/
copying  tmp_2409.14445/./figures/chn_CH3OH_

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# oMEGACat IV: Constraining Ages of Omega Centauri sub-giant branch stars with _HST_ and MUSE

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.13855-b31b1b.svg)](https://arxiv.org/abs/2409.13855)<mark>Appeared on: 2024-09-24</mark> -  _23 pages, 11 figures_

</div>
<div id="authors">

<mark>C. Clontz</mark>, et al. -- incl., <mark>M. Häberle</mark>, <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** We present age estimates for over 8100 sub-giant branch (SGB) stars in Omega Centauri ( $\omc$ ) to study its star formation history. Our large data set, which combines multi-wavelength _HST_ photometry with MUSE metallicities, provides an unprecedented opportunity to measure individual stellar ages. We do this by fitting each star's photometry and metallicity with theoretical isochrones, that are embedded with an empirical [ C+N+O ] - [ Fe/H ] relation specifically for $\omc$ . The bulk of the stars have ages between 13 and 10 Gyr, with the mean stellar age being 12.08 $\pm{0.01}$ Gyrs and the median age uncertainty being 0.68 Gyrs. From these ages we construct the most complete age-metallicity relation (AMR) for $\omc$ to date.  We find that the mean age of stars decreases with increasing metallicity and find two distinct streams in the age-metallicity plane, hinting at different star formation pathways. We derive an intrinsic spread in the ages of 0.75 $\pm{0.01}$ Gyr for the whole cluster, with the age spread showing a clear increase with metallicity. We verif $\newtext{y}$ the robustness of our age estimations by varying isochrone parameters and constraining our systematics. We find the C+N+O relation to be the most critical consideration for constraining the AMR. We also present the SGB chromosome map with age information. In the future, these stellar ages could be combined with chemical abundances to study age differences in subpopulations, and uncover the chemical evolution history of this massive nuclear star cluster.

</div>

<div id="div_fig1">

<img src="tmp_2409.13855/./figures/mean_ages_and_age_spreads_systematics.png" alt="Fig10" width="100%"/>

**Figure 10. -** (*left*) **Mean Age vs. [Fe/H] Systematics:** Our mean age (\em{left}) and age spread (\em{right}) as a function of metallicity with our fiducial results (black points) as well as systematic tests (colored points; marker labels shared with both panels).  In both panels errorbars show 1$\sigma$ uncertainty and gray bars along the top denote the [Fe/H] equal number bin edges. Generally, our systematics tests slightly affect the mean ages but not the trend with [Fe/H] except for the C+N+O abundance, which significantly impacts this measurement. The age spread is largely independent of any systematic effects.
 (*fig:age_and_age_spread_systematics*)

</div>
<div id="div_fig2">

<img src="tmp_2409.13855/./figures/amr_two_panels.png" alt="Fig8" width="100%"/>

**Figure 8. -** **The Age-Metallicity Relation:**(*left*) The age distribution of the SGB star sample in equal number ($ \rm N_* = 803$) metallicity bins. The colors correspond to the colormap of Figure 1. (*right*) The 2D histogram of stars in the age-metallicity plane is given by the colormap, where purple is less dense and yellow is denser. The median uncertainty is given by the black contour in the lower center. The Gaia-Sausage /Enceladus (GS/E) SFH  ([Limberg, Souza and Pérez-Villegas 2022]())  is shown by the orange line. \newtext{On the right side we show the full [Fe/H] histogram with each equal number bin region denoted by colored patches. On the top of both panels we show the full age histogram for the clean SGB sample.} (*fig:age_met_relation*)

</div>
<div id="div_fig3">

<img src="tmp_2409.13855/./figures/sgb_polygon_selection.png" alt="Fig6" width="100%"/>

**Figure 6. -** **SGB Selection \& Relevant Isochrones:** The left panel shows our initial SGB region selection colored by the MUSE metallicity. The middle panel shows stars with $-1.75 < {\rm[Fe/H]} < -1.65$ and isochrones (described in Section \ref{subsec:iso_models}) with [Fe/H] = -1.7  and ages ranging from 5 to 15 Gyr (top to bottom) overplotted in green. The solid lines are the solar scaled helium abundance isochrones and the dashed lines are the Y$=$0.40 models. The right panel is similar to the middle, now with ${\rm[Fe/H]} = -1.0$ and isochrones colored in red. Our selection utilizes the region where the models are the most similar. (*fig:sgb_region*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.13855"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\dij}[1]{\textcolor{blue}{#1}}$
$\newcommand{\dijcomm}[1]{\textcolor{blue}{DIJ: #1}}$
$\newcommand{\percbcm}{\mbox{~cm^{-3}}}$
$\newcommand{\persqcm}{\mbox{~cm^{-2}}}$
$\newcommand{\perbeam}{\mbox{beam^{-1}}}$
$\newcommand{\Jyperbeam}{\mbox{~Jy~beam^{-1}}}$
$\newcommand{\mJyperbeam}{\mbox{~mJy~beam^{-1}}}$
$\newcommand{\muJyperbeam}{\mbox{~\muJy~beam^{-1}}}$
$\newcommand{\kmpers}{\mbox{~km~s^{-1}}}$
$\newcommand{\vLSR}{v_\mathrm{LSR}}$
$\newcommand{\voff}{v_\mathrm{off}}$
$\newcommand{\frest}{f_\mathrm{rest}}$
$\newcommand{\Aij}{A_\mathrm{ij}}$
$\newcommand{\Eu}{E_\mathrm{u}}$
$\newcommand{\gu}{g_\mathrm{u}}$
$\newcommand{\vwidth}{\delta v}$
$\newcommand{\ii}{_2}$
$\newcommand{\iii}{_3}$
$\newcommand{\xviii}{^{18}}$
$\newcommand{\Trot}{T_\mathrm{rot}}$
$\newcommand{\Ntot}{N_\mathrm{tot}}$
$\newcommand{\Lsun}{L_\odot}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Lbol}{L_\mathrm{bol}}$
$\newcommand{\Tbol}{T_\mathrm{bol}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\brown}[1]{\textcolor{brown}{#1}}$
$\newcommand{\magenta}[1]{\textcolor{magenta}{#1}}$
$\newcommand{\purple}[1]{\textcolor{purple}{#1}}$
$\newcommand{\syhcomm}[1]{\textcolor{brown}{SYH: #1}}$
$\newcommand{\MDOTj}{\dot{M}_\mathrm{j}}$
$\newcommand{\MDOTEq}[2]{#1 \times 10^{#2}}$
$\newcommand{\MDOTEqUnit}[2]{#1 \times 10^{#2}~M\subsun\peryr}$
$\newcommand{\MDOTEqRangeUnit}[4]{#1 \times 10^{#2} -- #3 \times 10^{#4}~M\subsun\peryr}$
$\newcommand{\REVO}[1]{#1}$
$\newcommand{\REVI}[1]{\textbf{#1}}$
$\newcommand{\REVII}[1]{#1}$
$\newcommand{\REVIII}[1]{#1}$
$\newcommand{\ang}{Å}$
$\newcommand{\degpoint}{\mbox{^\circ\mskip-7.0mu. }}$
$\newcommand{\halpha}{\mbox{H\alpha}}$
$\newcommand{\hbeta}{\mbox{H\beta}}$
$\newcommand{\hgamma}{\mbox{H\gamma}}$
$\newcommand{\kms}{ km~s^{-1}}$
$\newcommand{\lya}{\mbox{Ly\alpha}}$
$\newcommand{\lyb}{\mbox{Ly\beta}}$
$\newcommand{\minpoint}{\mbox{'\mskip-4.7mu.\mskip0.8mu}}$
$\newcommand{\mv}{\mbox{m_{_V}}}$
$\newcommand{\Mv}{\mbox{M_{_V}}}$
$\newcommand{\peryr}{\mbox{\>\rm yr^{-1}}}$
$\newcommand{\secpoint}{\mbox{"\mskip-7.6mu. }}$
$\newcommand{\sqdeg}{\mbox{{\rm deg}^2}}$
$\newcommand{\squig}{\sim\!\!}$
$\newcommand{\subsun}{\mbox{_{\normalsize\odot}}}$
$\newcommand{\deg}{\hbox{^\circ}}$
$\newcommand{\sun}{\hbox{\odot}}$
$\newcommand{\earth}{\hbox{\oplus}}$
$\newcommand\lesssim{\mathrel{\hbox{\rlap{\hbox{$
$ \lower4pt\hbox{\sim}}}\hbox{<}}}}$
$\newcommand\gtrsim{\mathrel{\hbox{\rlap{\hbox{$
$ \lower4pt\hbox{\sim}}}\hbox{>}}}}$
$\newcommand{\sq}{\hbox{\rlap{\sqcap}\sqcup}}$
$\newcommand{\arcmin}{\hbox{^\prime}}$
$\newcommand{\arcs}{\hbox{^{\prime\prime}}}$
$\newcommand{\fd}{\hbox{.\!\!^{\rm d}}}$
$\newcommand{\fh}{\hbox{.\!\!^{\rm h}}}$
$\newcommand{\fm}{\hbox{.\!\!^{\rm m}}}$
$\newcommand{\fs}{\hbox{.\!\!^{\rm s}}}$
$\newcommand{\fdg}{\hbox{.\!\!^\circ}}$
$\newcommand{\farcm}{\hbox{.\mkern-4mu^\prime}}$
$\newcommand{\farcs}{\hbox{.\!\!^{\prime\prime}}}$
$\newcommand{\fp}{\hbox{.\!\!^{\scriptscriptstyle\rm p}}}$
$\newcommand{\micron}{\hbox{\mum}}$</div>



<div id="title">

# ALMASOP. The Localized and Chemically rich Features near the Bases of the Protostellar Jet in HOPS 87

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.14445-b31b1b.svg)](https://arxiv.org/abs/2409.14445)<mark>Appeared on: 2024-09-24</mark> -  _16 pages, 6+2 figures, accepted by ApJ_

</div>
<div id="authors">

S.-Y. Hsu, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** HOPS 87 is a Class 0 protostellar core known to harbor an extremely young bipolar outflow and a hot corino. We report the discovery of localized, chemically rich regions near the bases of the two-lobe bipolar molecular outflow in HOPS 87 containing molecules such as $H_2$ CO, $^{13}$ CS, $H_2$ S, OCS, and $CH_3$ OH, the simplest complex organic molecule (COM).The locations and kinematics suggest that these localized features are due to jet-driven shocks rather than being part of the hot corino region encasing the protostar.The COM compositions of the molecular gas in these jet-localized regions are relatively simpler than those in the hot corino zone.We speculate that this simplicity is due to either the liberation of ice with a less complex chemical history or the effects of shock chemistry.Our study highlights the dynamic interplay between the protostellar bipolar outflow, disk, inner core environment, and the surrounding medium, contributing to our understanding of molecular complexity in solar-like young stellar objects.

</div>

<div id="div_fig1">

<img src="tmp_2409.14445/./figures/mom0_transs.png" alt="Fig3" width="100%"/>

**Figure 3. -** 
Integrated intensity images (moment-0 maps) of selected molecular transitions: C$^{18}$O, CO, $CH_3$CHO, HCOOCH$_{3}$, H$_{2}$CO, CH$_{3}$OH, OCS, $^{13}$CS, and $H_2$S.
The origin of the coordinate system is the 1.3 mm continuum peak at [$\alpha_{2000}, \delta_{2000}$]$=$[05$^\mathrm{h}$35$^\mathrm{m}$23$^\mathrm{s}$\!\!.42, -05$◦$01$\arcmin$30$\arcs$\!\!.6], and the ranges of right ascension (x-axis) and declination (y-axis) span $\pm2$\arcs$$.
Each panel is labeled at the top with the corresponding chemical formula and the upper energy level of the transition shown in color.
The velocity ranges for integration are $\pm 40$ $\kmpers$ for CO and SiO, $\pm 3$ $\kmpers$ for $^{13}$CS, and $\pm 7.5$ $\kmpers$ for others.
The black dashed contours depict the 1.3 mm continuum, set at levels of [5, 10, 20, 40, 80, 160]$\sigma$, where $\sigma$ corresponds to 0.9$\mJyperbeam$(or 0.13 K in terms of brightness temperature).
The orange ellipse located at the bottom left corner indicate the beam sizes for the Cycle 6 ALMA program.
The green contours display the SiO integrated intensity map, highlighted at levels of [10, 25, 40]$\sigma$, where each $\sigma$ corresponds to 28$\mJyperbeam$\kmpers.
The distribution of the molecules in panels (e)--(i) are particularly noteworthy; these molecules are not typical outflow tracers but exhibit two-lobed distributions.
The white cross in panels (c) and (d) illustrate the peak position of the corresponding transition.
 (*fig:mom0_outflow*)

</div>
<div id="div_fig2">

<img src="tmp_2409.14445/./figures/PVtranss.png" alt="Fig6" width="100%"/>

**Figure 6. -** 
The PV diagrams of selected molecular transitions along $\REVI$I{the} outflow axis (position angle PA $=-4$◦ $\REVI$I{with} the center is at the 1.3 mm continuum peak.
The relative position $\REVI$I{offset}($\Delta p$) is from south ($\Delta p<0$\arcs$$) to north ($\Delta p>0$\arcs$$).
The upper left label in each panel show the chemical species and the upper energy $\Eu$ of the transitions.
Note that the diagrams are exported from the data cube of the full spectral window so the $\REVI$I{separated components} may $\REVI$I{come} from molecular transitions other than the labeled one in each label (e.g., the components at (-7.5$\kmpers$, 0$\arcs$) in panel (h) and at (-7.5$\kmpers$, 0$\arcs$) in panel (k).
The color maps and the brown contours are the PV diagram of the labeled transitions.
The grey contours represent the CO PV diagram.
The brown and grey contours are in steps of [5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80]$\sigma$.
The navy polyline, referred from the PV diagram of the $CH_3$OH 45 K transition, consists of two horizontal lines at $\Delta p=0$\farcs${3}$ and $-0$\farcs${2}$ and a $\REVI$I{diagonal} line moves  from $\Delta p=0$\farcs${3}$ to $-0$\farcs${2}$ at $\Delta v=0.75$\kmpers and $-0.75$\kmpers, respectively.
The diagrams are exported by CARTA 4.0 \citep[][]{2021Comrie_CARTA}.
The gap in CO indicates the missing flux at low velocities due to interferometry.
$\REVI$I{
The broadening of the velocity range localized at $\Delta p=0$\farcs${3}$ and $-0$\farcs${2}$ is significant in $CH_3$OH, $H_2$CO, OCS, and $H_2$S and tentatively observed in $^{13}$CS.
Additionally, the rapid growth of velocity $\Delta v$ with position offset $\Delta p$ described in 2001Lee_jet_wind_model is significant in SiO.
}
These characteristics suggest that these molecules are also tracing jet-driven bow shocks  (2001Lee_jet_wind_model) .
 (*fig:PVtranss*)

</div>
<div id="div_fig3">

<img src="tmp_2409.14445/./figures/chn_CH3OH_Cycle3.png" alt="Fig4" width="100%"/>

**Figure 4. -** 
The $CH_3$OH $5_{1,4}-4_{2,3}$ transition in Cycle 3 observations.
The top and bottom rows show the images at the blue- and red-shifted channels, respectively.
The labels in each panel shows the relative velocity ($\Delta v = v - v_\mathrm{LSR}$).
At $\Delta v=0$ $\kmpers$ the intensity map displays an arc-like emission pattern west of the center with an absorption feature toward the continuum peak.
At $|\Delta v|>0.68$ $\kmpers$, there is a single lobe ($\Delta \delta \sim 0$\farcs${3}$) on each of the northern and the southern sides of the arc ($\Delta \delta \sim 0$\farcs${2}$) seen at $\Delta v=0$ $\kmpers$.
 (*fig:chn_CH3OH_Cycle3*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.14445"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

191  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
